<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Teorema de la falsa posición</h2>
        <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/False_position_method.svg/702px-False_position_method.svg.png alt=middle style="height: 340px">
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [29]:
format long

function [xk,e]=false_pos_aux(footion, ck, xk, tol, i, iter, error, e)
    if footion(xk) * footion(ck) < 0
        if i >= iter || error <= tol
            xk=xk;
            e=e;
        else 
            x2 = xk - ((xk - ck) / (footion(xk) - footion(ck))) * footion(xk);
            error = abs(footion(x2));
            e = [e error];
            xk = false_pos_aux(footion, ck, x2, tol, i+1, iter, error, e);
            if xk == 0
                xk = false_pos_aux(footion, xk, x2, tol, i+1, iter, error, e);
            end         
        end
    else
        xk = 0;
        e = [];
    end
end


function [xk,e]=false_pos(f, x0, x1, tol, iter)
    footion = str2func(strcat('@(x)',f));
    if footion(x0) * footion(x1) < 0 && tol > 10^-16
        x2 = x1 - ((x1 - x0) / (footion(x1) - footion(x0))) * footion(x1);
        error = abs(footion(x2));
        [a,e1] = false_pos_aux(footion, x0, x2, tol, 0, iter, error, [error]);
        [b,e2] = false_pos_aux(footion, x1, x2, tol, 0, iter, error, [error]);
        if a
            xk = a;
            e = e1;
        else
            xk = b;
            e = e2;
        end
    else
        xk=0;
        e=[];
        display("error");
    end
end

[xk,e]=false_pos('exp(x)-2', 0, 3, 10^-12, 10)

xk =    6.613657180966892e-01
e =

   8.297854717577251e-01   6.743580602595696e-01



<h3 style="font-family: 'Courier New'; font-size:25px">Python</h3>

In [28]:
def calc_x(footion, xk, c):
    fx0 = float(footion.subs({'x': c}))
    fx1 = float(footion.subs({'x': xk}))
    x2 = float(xk - ((xk - c) / (fx1 - fx0)) * fx1)
    return x2

def false_pos(f, x0, x1, tol, iter):
    from sympy import sympify
    footion = sympify(f)
    x2=0

    fx=float(footion.subs({'x': x0}) * footion.subs({'x': x1}))
    if fx<0:
        x2=float(x1-(x1-x0)/(footion.subs({'x': x1})-footion.subs({'x': x0})) * footion.subs({'x': x1}))
    else:
        return 0
    error = tol + 1;
    a = x0
    b = x1
    c = x0
    e = []
    i = 0
    while error > tol and iter >= i:
        fx0=float(footion.subs({'x': c}))
        fx1=float(footion.subs({'x': x2}))
        if fx0*fx1<0:
            b=x2
            c=a
            x2=calc_x(footion, b, c)
            error=abs(x2-b)
        else:
            a=x2
            c=b
            x2=calc_x(footion, a, c)
            error=abs(x2-a)
        e.append(error)
        i+=1
        
    return x2, e

print(false_pos('exp(x)-2', 0, 3, 10^-12, 10))

1
2
3
4
5
6
7
8
9
10
11
(0.6931471805599453, [0.1247097354634738, 0.5410822614364288, 0.16675416623959127, 0.03456346249197284, 0.002402629410624413, 4.464562864703314e-05, 8.141340732681357e-07, 1.7842616273355816e-11, 0.0, 0.0, 0.0])
